In [1]:
# Append system path to include the config scripts
import sys
import os
from copy import deepcopy
import pickle

print('importing lanfactory')
import lanfactory

print('importing ssms')
import ssms

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from config import *

import torch
import config

import pandas as pd

importing lanfactory
importing ssms
importing lanfactory
importing ssms


In [2]:
import yaml

In [5]:
yaml.safe_load(open('network_training_config_af.yaml', 'rb'))

{'NETWORK_TYPE': 'cpn',
 'CPU_BATCH_SIZE': 128,
 'GPU_BATCH_SIZE': 512,
 'GENERATOR_APPROACH': 'lan',
 'OPTIMIZER_': 'adam',
 'N_EPOCHS': 20,
 'PROJECT_FOLDER': '/users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/',
 'MODEL': 'ddm',
 'layer_sizes': [[100, 100, 100, 1],
  [100, 100, 100, 100, 1],
  [100, 100, 100, 100, 100, 1],
  [120, 120, 120, 1],
  [120, 120, 120, 120, 1],
  [120, 120, 120, 120, 120, 1]],
 'activations': ['NETWORK_TYPE']}

In [4]:
ssms.config.model_config.keys()

dict_keys(['ddm', 'ddm_legacy', 'ddm_deadline', 'angle', 'weibull', 'levy', 'levy_angle', 'full_ddm', 'gamma_drift', 'gamma_drift_angle', 'ds_conflict_drift', 'ds_conflict_drift_angle', 'ornstein', 'ornstein_angle', 'ddm_sdv', 'race_3', 'race_no_bias_3', 'race_no_bias_angle_3', 'race_4', 'race_no_bias_4', 'race_no_bias_angle_4', 'lca_3', 'lca_no_bias_3', 'lca_no_bias_angle_3', 'lca_4', 'lca_no_bias_4', 'lca_no_bias_angle_4', 'ddm_par2', 'ddm_par2_no_bias', 'ddm_par2_conflict_gamma_no_bias', 'ddm_par2_angle_no_bias', 'ddm_par2_weibull_no_bias', 'ddm_seq2', 'ddm_seq2_no_bias', 'ddm_seq2_conflict_gamma_no_bias', 'ddm_seq2_angle_no_bias', 'ddm_seq2_weibull_no_bias', 'ddm_mic2_adj', 'ddm_mic2_adj_no_bias', 'ddm_mic2_adj_conflict_gamma_no_bias', 'ddm_mic2_adj_angle_no_bias', 'ddm_mic2_adj_weibull_no_bias', 'tradeoff_no_bias', 'tradeoff_angle_no_bias', 'tradeoff_weibull_no_bias', 'tradeoff_conflict_gamma_no_bias', 'glob', 'weibull_cdf', 'full_ddm2'])

# DATA GENERATOR CONFIGS

**Note:** 

Look into the [ssms]() package documentation to get a better idea about the kinds of configs that you need for different kinds of training data


### Define Metadata

**Note:**

ALL CAPITAL letter variables should be user specified.

In [4]:
# Specify model
# String name of a model which is accessible through the ssms package 
# (You can of course also use your own simulators, just make sure they are wrapped or designed in such a way
# that their returns match the style of the ssms training data generators)
MODEL = 'ddm' #'ddm_deadline' # 'ds_conflict_drift' 

# Specify data_generator parameters
N_SAMPLES = 20000 #200000 # How many samples per parameter set?
N_PARAMETER_SETS = 1000 #5000 # How many parameter sets in a single call?
DELTA_T = 0.001 # Delta_t to apply to the data (simulator speed depends on this)
N_TRAINING_SAMPLES_BY_PARAMETER_SET = 2000 # How many training samples to return for a single parameter set (Note that this is a different number from the number of samples to simulate per parameter set)
N_SUBRUNS = 20 # How many subruns to split the training data into (This is useful if you want to run the training on multiple machines in parallel)
CPN_ONLY = False # if true simulator will run faster but produce only choice probabilities

# proj folder
PROJECT_FOLDER = '/users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/' # Your project base folder (other folders are going to be generated automatically in here)

# What kind of likelihood approximator are we generating training data for?
generator_approach = 'lan' if CPN_ONLY is False else 'cpn_only' # 

### Use Metadata to specify arguments

In [5]:
# training data folder
training_data_folder = PROJECT_FOLDER + \
                         '/data/training_data/' + generator_approach + \
                            '/training_data_n_samples' + '_' + \
                                str(N_SAMPLES) + '/' + MODEL + '/'

# Where do you want to save the config file?
config_save_folder = PROJECT_FOLDER + '/data/config_files/data_generation/' + \
                        generator_approach + '/' + MODEL + '/'

# Specify arguments which you want to adjust in the data generator
data_generator_arg_dict = {
                           'output_folder': training_data_folder,
                           'dgp_list': MODEL,
                           'n_samples': N_SAMPLES,
                           'n_parameter_sets': N_PARAMETER_SETS,
                           'delta_t': DELTA_T,
                           'n_training_samples_by_parameter_set': N_TRAINING_SAMPLES_BY_PARAMETER_SET,
                           'n_subruns': N_SUBRUNS,
                           'cpn_only': CPN_ONLY,
                          }

# model_config_arg_dict = {'param_bounds': [[-2.5, 0.2, 0.1, 0.0],
#                                           [2.5, 2.2, 0.9, 2.0]]}
model_config_arg_dict = {}

# Name of the config file
data_config_save_name = 'nsim_' + str(data_generator_arg_dict['n_samples']) + \
                        '_dt_' + str(data_generator_arg_dict['delta_t']) + \
                        '_nps_' + str(data_generator_arg_dict['n_parameter_sets']) + \
                        '_npts_' + str(data_generator_arg_dict['n_training_samples_by_parameter_set']) + \
                        '.pickle'

### Generate the Config File

In [6]:
data_config_dict = make_data_generator_configs(model = MODEL,
                                               generator_approach = generator_approach,
                                               data_generator_arg_dict = data_generator_arg_dict,
                                               model_config_arg_dict = model_config_arg_dict,
                                               save_name = data_config_save_name,
                                               save_folder = config_save_folder)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/config_files
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/config_files/data_generation
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/config_files/data_generation/lan
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/config_files/data_generation/lan/ddm
Moving on...
Saved to: 
/users/afengler/data/proj_lan_pipeline/LAN_scripts//data/config_files/data_generation/lan/ddm/nsim_20000_dt_0.001_nps_1000_npts_2000.pickle


In [10]:
data_config_dict

{'config_dict': {'model_config': {'name': 'ddm',
   'params': ['v', 'a', 'z', 't'],
   'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]],
   'boundary': <function ssms.basic_simulators.boundary_functions.constant(t=0)>,
   'n_params': 4,
   'default_params': [0.0, 1.0, 0.5, 0.001],
   'hddm_include': ['z'],
   'nchoices': 2},
  'data_config': {'output_folder': '/users/afengler/data/proj_lan_pipeline/LAN_scripts//data/training_data/cpn_only/training_data_n_samples_20000/ddm/',
   'dgp_list': 'ddm',
   'n_samples': 20000,
   'n_parameter_sets': 1000,
   'n_parameter_sets_rejected': 100,
   'n_training_samples_by_parameter_set': 2000,
   'max_t': 20.0,
   'delta_t': 0.001,
   'pickleprotocol': 4,
   'n_cpus': 'all',
   'negative_rt_cutoff': -66.77497,
   'n_subruns': 20,
   'cpn_only': True}},
 'config_file_name': '/users/afengler/data/proj_lan_pipeline/LAN_scripts//data/config_files/data_generation/cpn_only/ddm/nsim_20000_dt_0.001_nps_1000_npts_2000.pickle'}

# NETWORK AND TRAIN CONFIGS

### Define Metadata

In [22]:
# network type
NETWORK_TYPE = 'cpn' # lan, choicep

# Backend dependent batch size (note that batch sizes for cpns can be much smaller than the ones for LANs)
# Data is passed to CPNs parameter-wise, whiel for LANs it is passed 'trial-wise' (inflating the total amount of training data for LANs by a factor of ~ 1000)
CPU_BATCH_SIZE = 128 # lan: 1000, cpn: 128  # Batch size if cpu backend is detected (note that training success can very well depend on batch size, this parameter doesn't just affect the training speed)
GPU_BATCH_SIZE = 512 # lan: 50000, cpn: 1024 # Batch size if gpu backend is detected

# What kind of training data?
# Note: If you have `lan` data you can also train a `cpn` on it
# The option is here to allow for training a `cpn` on `cpn` data only, since this is somewhat cheaper to produce
GENERATOR_APPROACH = 'lan'

# Which optimizer to choose?
OPTIMIZER_ = 'adam' # 'adam', 'sgd'

# How many epochs to train?
N_EPOCHS = 20 # reasonable defaults --> lan: 20, cpn: 50

# Where do you want to save config files?
network_train_config_save_folder = PROJECT_FOLDER + '/data/config_files/network/' + \
                                       NETWORK_TYPE + '/' + MODEL + '/'



# Specify training data folder:
training_data_folder = PROJECT_FOLDER + \
                           '/data/training_data/' + GENERATOR_APPROACH + '/' + \
                               'training_data_n_samples' +  \
                                '_' + str(N_SAMPLES) + '/' + MODEL

# Specify the name of the config file
network_train_config_save_name = 'train_config' + \
                                     '_opt_' + OPTIMIZER_ + \
                                        '_n_' + str(N_SAMPLES) + \
                                            '_dt_' + str(DELTA_T) + \
                                                '_nps_' + str(N_PARAMETER_SETS) + \
                                                    '_npts_' + str(N_TRAINING_SAMPLES_BY_PARAMETER_SET) + \
                                                        '_architecture_search.pickle'




# Train output type specifies what the network output node
# 'represents' (e.g. log-probabilities / logprob, logits, probabilities / prob)

# Specifically for cpn, we train on logit outputs for numerical stability, then transform outputs
# to log-probabilities when running the model in evaluation / inference mode 
train_output_type_dict = {'lan': 'logprob',
                          'cpn': 'logits',
                          'cpn_bce': 'prob'}

# Last layer activation depending on train output type
output_layer_dict = {'logits': 'linear',
                     'logprob': 'linear',
                     'prob': 'sigmoid'}

# LOSS 
# 'bce' (for binary-cross-entropy), use when train output is 'prob'
# 'bcelogit' (for binary-cross-entropy with inputs representing logits) use when train output type is 'logits', (this is standard for cpns)
# 'huber' (usually) used when train output is 'logprob'

train_loss_dict = {'logprob': 'huber',
                   'logits': 'bcelogit',
                   'prob': 'bce'}

# Network architectures
layer_sizes = [[100, 100, 100, 1], [100, 100, 100, 100, 1], [100, 100, 100, 100, 100, 1],
               [120, 120, 120, 1], [120, 120, 120, 120, 1], [120, 120, 120, 120, 120, 1]] #,
#                [150, 150, 150, 1], [150, 150, 150, 150, 1], [150, 150, 150, 150, 150, 1]]

activations = [['tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]], 
               ['tanh', 'tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]], 
               ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]],
               ['tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]], 
               ['tanh', 'tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]], 
               ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', output_layer_dict[train_output_type_dict[NETWORK_TYPE]]]
              ]

weight_decays = [0.0]

# Train / validations split
train_val_split = [0.98, 0.98, 0.98,
                   0.98, 0.98, 0.98]

# Number is set to 10000 here (an upper bound), for training on all available data (usually roughly 300 files, but has never been more than 1000)
# For numerical experiments, one may want to artificially constraint the number of training files to teest the impact on network performance
n_training_files = [10000]

# 
network_arg_dict = {'train_output_type': train_output_type_dict[NETWORK_TYPE],
                    'network_type': NETWORK_TYPE}


data_key_dict = {'lan': {'features_key': 'data', 
                         'label_key': 'labels'},
                 'cpn': {'features_key': 'thetas',
                         'label_key': 'choice_p'}
                }

# initial train_arg_dict
# refined in for loop in next cell
train_arg_dict = {'n_epochs': N_EPOCHS,
                  'loss': train_loss_dict[train_output_type_dict[NETWORK_TYPE]],
                  'optimizer': OPTIMIZER_,
                  'train_output_type': train_output_type_dict[NETWORK_TYPE],
                  # 'n_training_files': n_training_files[j],
                  # 'train_val_split': train_val_split[i],
                  'cpu_batch_size': CPU_BATCH_SIZE,
                  'gpu_batch_size': GPU_BATCH_SIZE,
                  'shuffle_files': True,
                  'label_lower_bound': np.log(1e-7),
                  # 'weight_decay': weight_decays[k],
                  'learning_rate': 0.001,
                  'features_key': data_key_dict[NETWORK_TYPE]['features_key'],
                  'label_key': data_key_dict[NETWORK_TYPE]['label_key'],
                  'save_history': True,
                  'lr_scheduler': 'reduce_on_plateau',
                  'lr_scheduler_params': {'factor': 0.1,
                                          'patience': 2,
                                          'threshold': 0.001,
                                          'min_lr': 0.00000001,
                                          'verbose': True,
                                         }
                  }

In [9]:
basic_config = yaml.safe_load(open('network_training_config_af.yaml', 'rb'))['cpn']


# Where do you want to save config files?
network_train_config_save_folder = basic_config['PROJECT_FOLDER'] + '/data/config_files/network/' + \
                                       basic_config['NETWORK_TYPE'] + '/' + basic_config['MODEL'] + '/'


# Specify training data folder:
training_data_folder = basic_config['PROJECT_FOLDER'] + \
                           '/data/training_data/' + basic_config['GENERATOR_APPROACH'] + '/' + \
                               'training_data_n_samples' +  \
                                '_' + str(N_SAMPLES) + '/' + basic_config['MODEL']

# Specify the name of the config file
network_train_config_save_name = 'train_config' + \
                                     '_opt_' + basic_config['OPTIMIZER_'] + \
                                        '_n_' + str(N_SAMPLES) + \
                                            '_dt_' + str(DELTA_T) + \
                                                '_nps_' + str(N_PARAMETER_SETS) + \
                                                    '_npts_' + str(N_TRAINING_SAMPLES_BY_PARAMETER_SET) + \
                                                        '_architecture_search.pickle'

# Train output type specifies what the network output node
# 'represents' (e.g. log-probabilities / logprob, logits, probabilities / prob)

# Specifically for cpn, we train on logit outputs for numerical stability, then transform outputs
# to log-probabilities when running the model in evaluation / inference mode 
train_output_type_dict = {'lan': 'logprob',
                          'cpn': 'logits',
                          'cpn_bce': 'prob'}

# Last layer activation depending on train output type
output_layer_dict = {'logits': 'linear',
                     'logprob': 'linear',
                     'prob': 'sigmoid'}

# LOSS 
# 'bce' (for binary-cross-entropy), use when train output is 'prob'
# 'bcelogit' (for binary-cross-entropy with inputs representing logits) use when train output type is 'logits', (this is standard for cpns)
# 'huber' (usually) used when train output is 'logprob'

train_loss_dict = {'logprob': 'huber',
                   'logits': 'bcelogit',
                   'prob': 'bce'}

data_key_dict = {'lan': {'features_key': 'data', 
                         'label_key': 'labels'},
                 'cpn': {'features_key': 'thetas',
                         'label_key': 'choice_p'}
                }

# Network architectures
layer_sizes = basic_config['layer_sizes']


activations = basic_config['activations']

# Append last layer (type of layer depends on type of network as per train_output_type_dict dictionary above)
activations = [act_tmp.append(output_layer_dict[train_output_type_dict[basic_config['NETWORK_TYPE']]]) for act_tmp in activations]

# Train / validations split
train_val_split = basic_config['train_val_split']
weight_decay = basic_config['weight_decay']
# train_val_split = [0.98, 0.98, 0.98,
#                    0.98, 0.98, 0.98]

# Number is set to 10000 here (an upper bound), for training on all available data (usually roughly 300 files, but has never been more than 1000)
# For numerical experiments, one may want to artificially constraint the number of training files to teest the impact on network performance
n_training_files = basic_config['n_training_files']

network_arg_dict = {'train_output_type': train_output_type_dict[basic_config['NETWORK_TYPE']],
                    'network_type': basic_config['NETWORK_TYPE']}

# initial train_arg_dict
# refined in for loop in next cell
train_arg_dict_new = {'n_epochs': basic_config['N_EPOCHS'],
                      'loss': train_loss_dict[train_output_type_dict[basic_config['NETWORK_TYPE']]],
                      'optimizer': basic_config['OPTIMIZER_'],
                      'train_output_type': train_output_type_dict[basic_config['NETWORK_TYPE']],
                      'n_training_files': n_training_files,
                      'train_val_split': train_val_split,
                      'weight_decay': weight_decay,
                      'cpu_batch_size': basic_config['CPU_BATCH_SIZE'],
                      'gpu_batch_size': basic_config['GPU_BATCH_SIZE'],
                      'shuffle_files': basic_config['SHUFFLE'],
                      'label_lower_bound': eval(basic_config['labels_lower_bound']),
                      # 'weight_decay': weight_decays[k],
                      'learning_rate': basic_config['learning_rate'],
                      'features_key': data_key_dict[basic_config['NETWORK_TYPE']]['features_key'],
                      'label_key': data_key_dict[basic_config['NETWORK_TYPE']]['label_key'],
                      'save_history': True,
                      'lr_scheduler': basic_config['lr_scheduler'],
                      'lr_scheduler_params': basic_config['lr_scheduler_params']
                      }

In [21]:
basic_config

{'NETWORK_TYPE': 'cpn',
 'CPU_BATCH_SIZE': 128,
 'GPU_BATCH_SIZE': 512,
 'GENERATOR_APPROACH': 'lan',
 'OPTIMIZER_': 'adam',
 'N_EPOCHS': 20,
 'PROJECT_FOLDER': '/users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/',
 'MODEL': 'ddm',
 'layer_sizes': [[100, 100, 100, 1],
  [100, 100, 100, 100, 1],
  [100, 100, 100, 100, 100, 1],
  [120, 120, 120, 1],
  [120, 120, 120, 120, 1],
  [120, 120, 120, 120, 120, 1]],
 'activations': [['tanh', 'tanh', 'tanh', 'linear'],
  ['tanh', 'tanh', 'tanh', 'tanh', 'linear'],
  ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
  ['tanh', 'tanh', 'tanh', 'linear'],
  ['tanh', 'tanh', 'tanh', 'tanh', 'linear'],
  ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear']],
 'weight_decays': [0.0],
 'train_val_split': [0.98, 0.98, 0.98, 0.98, 0.98, 0.98],
 'n_training_files': [10000],
 'labels_lower_bound': 'np.log(1e-7)',
 'learning_rate': 0.001,
 'lr_scheduler': 'reduce_on_plateau',
 'lr_scheduler_params': {'factor': '0.1,',
  'patience': '2,',
  'thresho

In [28]:
train_arg_dict == train_arg_dict_new

True

In [25]:
train_arg_dict

{'n_epochs': 20,
 'loss': 'bcelogit',
 'optimizer': 'adam',
 'train_output_type': 'logits',
 'cpu_batch_size': 128,
 'gpu_batch_size': 512,
 'shuffle_files': True,
 'label_lower_bound': -16.11809565095832,
 'learning_rate': 0.001,
 'features_key': 'thetas',
 'label_key': 'choice_p',
 'save_history': True,
 'lr_scheduler': 'reduce_on_plateau',
 'lr_scheduler_params': {'factor': 0.1,
  'patience': 2,
  'threshold': 0.001,
  'min_lr': 1e-08,
  'verbose': True}}

In [26]:
train_arg_dict_new

{'n_epochs': 20,
 'loss': 'bcelogit',
 'optimizer': 'adam',
 'train_output_type': 'logits',
 'cpu_batch_size': 128,
 'gpu_batch_size': 512,
 'shuffle_files': True,
 'label_lower_bound': -16.11809565095832,
 'learning_rate': 0.001,
 'features_key': 'thetas',
 'label_key': 'choice_p',
 'save_history': True,
 'lr_scheduler': 'reduce_on_plateau',
 'lr_scheduler_params': {'factor': '0.1,',
  'patience': '2,',
  'threshold': 0.001,
  'min_lr': 1e-08,
  'verbose': True}}

### Create the Config file


Here you actually create the config file for a network training run. 

This will be created as a dictionary with high level keys `[0, 1, 2, ...]`, to facilitate batch jobs that index into the specific key and run training
on the respective network.

Under each numerical `key` you will find a `network_config` and a `train_config` which jointly specify the metadata for a training run.

**Note:**

Network training runs pre-suppose training data!


In [14]:
# Loop objects
config_dict = {}
train_arg_dict = train_arg_dict_new

cnt = 0
for i in range(len(layer_sizes)):    
    # Specify the arguments which you want to adjust in the network and train configs
    # For details check: lanfactory.config.network_config_mlp
    #                    lanfactory.config.train_config_mlp

    network_arg_dict['layer_sizes'] = layer_sizes[i]
    network_arg_dict['activations'] = activations[i]
    
    #train_arg_dict['n_training_files'] = n_training_files[j]
    #train_arg_dict['train_val_split'] = train_val_split[i]
    #train_arg_dict['weight_decay'] = weight_decays[k]

    config_dict[cnt] = make_train_network_configs(training_data_folder=training_data_folder,
                                                  save_folder = network_train_config_save_folder,
                                                  train_val_split=train_val_split,
                                                  network_arg_dict = deepcopy(network_arg_dict),
                                                  train_arg_dict = deepcopy(train_arg_dict),
                                                  save_name = None)

    cnt += 1

print('In total, ',
          len(list(config_dict.keys())),
              ' different networks will be trained with this config file')

print('Now saving')

# Create save_folder if not already there
lanfactory.utils.try_gen_folder(folder = network_train_config_save_folder,
                                allow_abs_path_folder_generation = True)
                
# pickle.dump(config_dict, 
#             open(network_train_config_save_folder + network_train_config_save_name, 'wb'))
# print(network_train_config_save_folder + network_train_config_save_name)

In total,  6  different networks will be trained with this config file
Now saving
Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal
Creating it...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/data
Creating it...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/data/config_files
Creating it...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/data/config_files/network
Creating it...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/data/config_files/network/cpn
Creating it...
Did not find folder:  /users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal/data/config_files/network/cpn/ddm
Creating it...


In [16]:
train_arg_dict_new

{'n_epochs': 50,
 'loss': 'bcelogit',
 'optimizer': 'adam',
 'train_output_type': 'logits',
 'n_training_files': 10000,
 'train_val_split': 0.98,
 'weight_decay': 0.0,
 'cpu_batch_size': 128,
 'gpu_batch_size': 512,
 'shuffle_files': True,
 'label_lower_bound': -16.11809565095832,
 'learning_rate': 0.001,
 'features_key': 'thetas',
 'label_key': 'choice_p',
 'save_history': True,
 'lr_scheduler': 'reduce_on_plateau',
 'lr_scheduler_params': {'factor': 0.1,
  'patience': 2,
  'threshold': 0.001,
  'min_lr': 1e-08,
  'verbose': True}}

In [15]:
config_dict[0]

{'config_dict': {'network_config': {'layer_sizes': [100, 100, 100, 1],
   'activations': None,
   'train_output_type': 'logits',
   'network_type': 'cpn'},
  'train_config': {'cpu_batch_size': 128,
   'gpu_batch_size': 512,
   'n_epochs': 50,
   'optimizer': 'adam',
   'learning_rate': 0.001,
   'lr_scheduler': 'reduce_on_plateau',
   'lr_scheduler_params': {'factor': 0.1,
    'patience': 2,
    'threshold': 0.001,
    'min_lr': 1e-08,
    'verbose': True},
   'weight_decay': 0.0,
   'loss': 'bcelogit',
   'save_history': True,
   'train_output_type': 'logits',
   'n_training_files': 10000,
   'train_val_split': 0.98,
   'shuffle_files': True,
   'label_lower_bound': -16.11809565095832,
   'features_key': 'thetas',
   'label_key': 'choice_p'},
  'training_data_folder': '/users/afengler/data/proj_lan_pipeline/LAN_pipeline_minimal//data/training_data/lan/training_data_n_samples_20000/ddm',
  'train_val_split': 0.98},
 'config_file_name': None}

In [11]:
x = pickle.load(open(network_train_config_save_folder + network_train_config_save_name, 'rb'))